In [6]:
import pandas as pd


## Get data from Johns Hopkins

In [7]:
#!git clone https://github.com/CSSEGISandData/COVID-19.git 
!cd COVID-19 && git pull origin master

From https://github.com/CSSEGISandData/COVID-19
 * branch            master     -> FETCH_HEAD
Already up to date.


In [8]:
df_corona_temp = pd.read_csv("./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")

### retreat data structure

In [9]:
df_corona_temp = df_corona_temp.groupby(by="Country/Region").sum()
df_corona_temp.drop(['Lat','Long'], axis=1, inplace=True)
df_corona_temp.fillna(0, inplace=True)
df_corona_temp.index.name = None
df_corona_temp.columns = pd.to_datetime(df_corona_temp.columns)
df_corona_temp.rename(index={'Mainland China':'China', 'Korea, South': 'South Korea'},inplace=True)


In [10]:
df_corona_temp.shape

(143, 53)

### Transform colums as lines

In [11]:
serie_corona = df_corona_temp.stack()

In [12]:
# back to df 
df_corona = pd.DataFrame(serie_corona)

df_corona.reset_index(inplace=True)
df_corona.columns = ['country', 'date','cases']

In [13]:
# Checking data 
df_corona.head()

,country,date,cases
0,Afghanistan,2020-01-22,0
1,Afghanistan,2020-01-23,0
2,Afghanistan,2020-01-24,0
3,Afghanistan,2020-01-25,0
4,Afghanistan,2020-01-26,0


## UN Data on population and density

### Alignment of data

In [14]:
def align_country_names(df):
    ''' Align country names with CSSE reference'''
    output = df.rename(columns={'Region, subregion, country or area *' : 'Country'})
    output.set_index("Country", inplace=True)
    output.rename(index={'Republic of Korea' : 'South Korea', 'Iran (Islamic Republic of)': 'Iran', 'United States of America' : 'US', 'China, Taiwan Province of China' :'Taiwan','Russian Federation':'Russia','Viet Nam':'Vietnam'}, inplace=True)
    # focus only at country level data
    output = output[output['Type'] == 'Country/Area']
    # return 2020 data olny
    return output['2020']


### Population

In [15]:
df_wp_temp = pd.read_excel("./UN data/population_world.xlsx", sheet_name = 'ESTIMATES', skiprows= range(0,16))

In [16]:
serie_wp = align_country_names(df_wp_temp)
#check data
serie_wp

Country
Burundi                      11890.8
Comoros                      869.595
Djibouti                     988.002
Eritrea                      3546.43
Ethiopia                      114964
                              ...   
Bermuda                       62.273
Canada                       37742.2
Greenland                     56.772
Saint Pierre and Miquelon      5.795
US                            331003
Name: 2020, Length: 235, dtype: object

### Density data

In [17]:
serie_density = align_country_names(pd.read_excel("./UN data/WPP2019_POP_F06_POPULATION_DENSITY.xlsx", sheet_name = 'ESTIMATES', skiprows= range(0,16)))

In [18]:
# Check data
serie_density

Country
Burundi                       463.037
Comoros                       467.273
Djibouti                       42.623
Eritrea                       35.1131
Ethiopia                      114.964
                               ...   
Bermuda                       1245.46
Canada                        4.15045
Greenland                    0.138316
Saint Pierre and Miquelon     25.1957
US                            36.1854
Name: 2020, Length: 235, dtype: object

### Country infos
Crossing population and density and calculating size

In [19]:
df_1 = pd.DataFrame(serie_wp)
df_1.columns = ['population']
df_2 = pd.DataFrame(serie_density)
df_2.columns =['density']

In [20]:
# join two series as df
df_country_info = df_1.join(df_2)
# Calculate size of countries
df_country_info['size'] =  df_country_info['population']/df_country_info['density'] * 1000

In [21]:
# Cehcking data
df_country_info

,population,density,size
Country,,,
Burundi,11890.8,463.037,25680
Comoros,869.595,467.273,1861
Djibouti,988.002,42.623,23180
Eritrea,3546.43,35.1131,101000
Ethiopia,114964,114.964,1e+06
...,...,...,...
Bermuda,62.273,1245.46,50
Canada,37742.2,4.15045,9.09351e+06
Greenland,56.772,0.138316,410450


### Saving the UN data

In [22]:
df_country_info.to_csv('country_info.csv')

## Cross data CSSE and pop / density / size

In [23]:
df_dataset = df_corona.join(df_country_info, how='left',on='country')

In [24]:
df_dataset

,country,date,cases,population,density,size
0,Afghanistan,2020-01-22,0,38928.3,59.6274,652860
1,Afghanistan,2020-01-23,0,38928.3,59.6274,652860
2,Afghanistan,2020-01-24,0,38928.3,59.6274,652860
3,Afghanistan,2020-01-25,0,38928.3,59.6274,652860
4,Afghanistan,2020-01-26,0,38928.3,59.6274,652860
...,...,...,...,...,...,...
7574,occupied Palestinian territory,2020-03-10,0,NaN,NaN,NaN
7575,occupied Palestinian territory,2020-03-11,0,NaN,NaN,NaN
7576,occupied Palestinian territory,2020-03-12,0,NaN,NaN,NaN
7577,occupied Palestinian territory,2020-03-13,0,NaN,NaN,NaN


In [25]:
## Calculate the age of crisis

In [26]:
ratio_crisis = 0.5

In [27]:
df_dataset['ratio'] = df_dataset['cases'] / df_dataset['population'] * 100

In [28]:
# check data
df_dataset

,country,date,cases,population,density,size,ratio
0,Afghanistan,2020-01-22,0,38928.3,59.6274,652860,0
1,Afghanistan,2020-01-23,0,38928.3,59.6274,652860,0
2,Afghanistan,2020-01-24,0,38928.3,59.6274,652860,0
3,Afghanistan,2020-01-25,0,38928.3,59.6274,652860,0
4,Afghanistan,2020-01-26,0,38928.3,59.6274,652860,0
...,...,...,...,...,...,...,...
7574,occupied Palestinian territory,2020-03-10,0,NaN,NaN,NaN,NaN
7575,occupied Palestinian territory,2020-03-11,0,NaN,NaN,NaN,NaN
7576,occupied Palestinian territory,2020-03-12,0,NaN,NaN,NaN,NaN
7577,occupied Palestinian territory,2020-03-13,0,NaN,NaN,NaN,NaN


In [29]:
### calculate ref data for crisis

In [30]:
serie_epidemic_start_date = df_dataset[df_dataset['ratio'] > ratio_crisis].groupby('country').min()['date']

In [31]:
df_epidemic_start = pd.DataFrame(serie_epidemic_start_date)
df_epidemic_start.columns = ['start_date']

In [32]:
### add start date to dataset

In [33]:
df_dataset = df_dataset.join(df_epidemic_start, how='right', on='country')

In [34]:
df_dataset['age'] = (df_dataset['date'] - df_dataset['start_date']).dt.days


In [35]:
#check data
df_dataset

,country,date,cases,population,density,size,ratio,start_date,age
53,Albania,2020-01-22,0,2877.8,105.029,27400,0,2020-03-12,-50
54,Albania,2020-01-23,0,2877.8,105.029,27400,0,2020-03-12,-49
55,Albania,2020-01-24,0,2877.8,105.029,27400,0,2020-03-12,-48
56,Albania,2020-01-25,0,2877.8,105.029,27400,0,2020-03-12,-47
57,Albania,2020-01-26,0,2877.8,105.029,27400,0,2020-03-12,-46
...,...,...,...,...,...,...,...,...,...
7362,United Kingdom,2020-03-10,384,67886,280.602,241930,0.565654,2020-03-10,0
7363,United Kingdom,2020-03-11,459,67886,280.602,241930,0.676133,2020-03-10,1
7364,United Kingdom,2020-03-12,459,67886,280.602,241930,0.676133,2020-03-10,2
7365,United Kingdom,2020-03-13,801,67886,280.602,241930,1.17992,2020-03-10,3


## Saving final data

In [36]:
df_dataset.to_csv('dataset.csv')